In [5]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SBA for CompanyABC').getOrCreate()
df = spark.read.load("C:/PE/CompanyABC_stock.csv", format="csv", header = True, inferSchema = True)

In [6]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [7]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [8]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [9]:
df.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

In [12]:
df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [15]:
df_HV = df.withColumn('HV Ratio', df['High'] / df['Volume'])
df_HV.show()


+----------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|            HV Ratio|
+----------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|7.367338463826307E-6|
|2012-01-09|         59.029999|   

In [20]:
from pyspark.sql import functions as Fc
df_HV.orderBy(Fc.desc("High")).show(1)

+----------+---------+---------+-----+---------+-------+---------+--------------------+
|      Date|     Open|     High|  Low|    Close| Volume|Adj Close|            HV Ratio|
+----------+---------+---------+-----+---------+-------+---------+--------------------+
|2015-01-13|90.800003|90.970001|88.93|89.309998|8215400|83.825448|1.107310672639189...|
+----------+---------+---------+-----+---------+-------+---------+--------------------+
only showing top 1 row



In [33]:
Q_1_5 = df_HV.select(Fc.avg("Close")).first()
print(Q_1_5)

Row(avg(Close)=72.38844998012726)


In [34]:
Q_1_6_max = df_HV.agg(Fc.max("Volume")).first()[0]
Q_1_6_min = df_HV.agg(Fc.min("Volume")).first()[0]

print(Q_1_6_max)
print(Q_1_6_min)

80898100
2094900


In [42]:
Q_1_7 = df_HV.filter(Fc.col("Close") < 70).count()
print(Q_1_7)

397


In [43]:
hdays = df_HV.filter(Fc.col("High") > 80).count()
totdays = df_HV.count()
Q_1_8 = (hdays / totdays) * 100

print(Q_1_8)


9.141494435612083


In [44]:
df.write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/CompanyABC_DB") \
    .option("dbtable", "CompanyABC_DB.stocks") \
    .option("user", "root") \
    .option("password", "password") \
    .save()
    

Section Two

In [54]:
feb_df = spark.read.load("C:/PE/Sales_February_2019.csv", format="csv", header = True, inferSchema = True)            
apr_df = spark.read.load("C:/PE/Sales_April_2019.csv", format="csv", header = True, inferSchema = True)
sales_df = feb_df.union(apr_df)

In [55]:
sales_df.printShema()

['Order ID',
 'Product',
 'Quantity Ordered',
 'Price Each',
 'Order Date',
 'Purchase Address']

In [57]:
sales_df.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



In [58]:
sales_df.describe().show()

+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|summary|          Order ID|     Product|   Quantity Ordered|        Price Each|    Order Date|    Purchase Address|
+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|  count|             30275|       30328|              30275|             30275|         30328|               30328|
|   mean|173816.63355904212|        null| 1.1239966969446737| 183.7621383980421|          null|                null|
| stddev|14902.179833784818|        null|0.43431295924749175|328.79347229907665|          null|                null|
|    min|            150502|20in Monitor|                  1|              2.99|02/01/19 01:51|1 14th St, New Yo...|
|    max|            194094|      iPhone|                  7|            1700.0|    Order Date|    Purchase Address|
+-------+------------------+------------+-------------------+---

In [59]:
sales_df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  150502|              iPhone|               1|     700.0|02/18/19 01:35|866 Spruce St, Po...|
|  150503|AA Batteries (4-p...|               1|      3.84|02/13/19 07:24|18 13th St, San F...|
|  150504|27in 4K Gaming Mo...|               1|    389.99|02/18/19 09:46|52 6th St, New Yo...|
|  150505|Lightning Chargin...|               1|     14.95|02/02/19 16:47|129 Cherry St, At...|
|  150506|AA Batteries (4-p...|               2|      3.84|02/28/19 20:32|548 Lincoln St, S...|
|  150507|Lightning Chargin...|               1|     14.95|02/24/19 18:50|387 12th St, Aust...|
|  150508|AA Batteries (4-p...|               1|      3.84|02/21/19 19:26|622 Center St, Sa...|
|  150509|Apple Airpods Hea...|         

In [60]:
sales_df = sales_df.withColumn("Total Price", Fc.col("Price Each") * Fc.col("Quantity Ordered"))
sales_df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+-----------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|Total Price|
+--------+--------------------+----------------+----------+--------------+--------------------+-----------+
|  150502|              iPhone|               1|     700.0|02/18/19 01:35|866 Spruce St, Po...|      700.0|
|  150503|AA Batteries (4-p...|               1|      3.84|02/13/19 07:24|18 13th St, San F...|       3.84|
|  150504|27in 4K Gaming Mo...|               1|    389.99|02/18/19 09:46|52 6th St, New Yo...|     389.99|
|  150505|Lightning Chargin...|               1|     14.95|02/02/19 16:47|129 Cherry St, At...|      14.95|
|  150506|AA Batteries (4-p...|               2|      3.84|02/28/19 20:32|548 Lincoln St, S...|       7.68|
|  150507|Lightning Chargin...|               1|     14.95|02/24/19 18:50|387 12th St, Aust...|      14.95|
|  150508|AA Batteries (4-p.

In [61]:
sales_df.write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost/CompanyABC_DB") \
    .option("dbtable", "CompanyABC_DB.sales") \
    .option("user", "root") \
    .option("password", "password") \
    .save()
    

In [62]:
sales_df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+-----------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|Total Price|
+--------+--------------------+----------------+----------+--------------+--------------------+-----------+
|  150502|              iPhone|               1|     700.0|02/18/19 01:35|866 Spruce St, Po...|      700.0|
|  150503|AA Batteries (4-p...|               1|      3.84|02/13/19 07:24|18 13th St, San F...|       3.84|
|  150504|27in 4K Gaming Mo...|               1|    389.99|02/18/19 09:46|52 6th St, New Yo...|     389.99|
|  150505|Lightning Chargin...|               1|     14.95|02/02/19 16:47|129 Cherry St, At...|      14.95|
|  150506|AA Batteries (4-p...|               2|      3.84|02/28/19 20:32|548 Lincoln St, S...|       7.68|
|  150507|Lightning Chargin...|               1|     14.95|02/24/19 18:50|387 12th St, Aust...|      14.95|
|  150508|AA Batteries (4-p.